<a href="https://colab.research.google.com/github/nonmetal/ssodam-qa-generator/blob/main/ssodam_crawler_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup
import time

In [3]:
# configuration
main_url = "http://www.ssodam.com/"
login_url = "http://www.ssodam.com/auth"
board_url_base = "http://www.ssodam.com/board/4/"  # board url without page number
content_url_base = "http://www.ssodam.com/content/"  # content url without post number

In [4]:
class Contents:
    def __init__(self, feed, title="", comments=""):
        if isinstance(feed, str):
            (postid, title, comments) = feed.strip().split("\t")
        else:
            postid = feed if isinstance(feed, int) else 0

        self.postid = int(postid)
        self.title = title
        self.comments = comments

    def __str__(self):
        try:
            return "%d\t%s\t%s" % (
                self.postid, self.title, self.comments)
        except UnicodeDecodeError:
            return "%d\t%s\t%s" % (
                self.postid, self.title, self.comments)

In [7]:
import re
data = open("/content/raws.txt", "r", encoding='utf-8')
sent = data.readlines()
array = []
for i in range(len(sent)):
    array.append(re.match('\d+',sent[i]).group())
len(array)

16949

In [11]:
newarr = array[10000:12000]
#2021-10-05 array 0~5000
#2021-10-05 array 5000~10000
newarr[0]

'169881'

In [12]:
session = requests.session()
print("Login Attempt")
#username, password = input("\tid: "), input("\tpassword: ")
csrftoken = session.get(main_url).cookies["csrftoken"]
payload = {
    "id": 'jyjoon',
    "password": 'p970801k',
    "auto": "false",
    "csrfmiddlewaretoken": csrftoken
}
result = session.post(login_url, data=payload, headers={"Referer": main_url})
if result.status_code != 200:
    print("Login Failure")
else:
    print("Login Success")

Login Attempt
Login Success


In [ ]:
# Figure out announcements
announcements = "911221"

# crawl each page and collect data
f_posts = open("/content/posts.txt", "w")
posts = []
post_number = newarr
counter = 1
print("Crawl Start")
for page in post_number:
    postid = int(page)
    content_url = content_url_base + page
    result = session.get(content_url)
    soup = BeautifulSoup(result.text, "lxml")
    title = soup.find_all("div", "board-title")[0].text
    for td in soup.find_all("div", "mverdana comment-content"):
        try:
            comments = []
            temp = td.text
            comments.append(temp)
        except (ValueError, IndexError):
            continue
        posts.append(Contents(postid, title, comments[0]))
        f_posts.write(str(posts[-1]) + "\n")
    print("\tReading page %d (%.2f%%)" % (
        postid, 100.0 * (counter - 1) / (len(post_number) - 1)))
    counter = counter + 1
    time.sleep(2)

print("Crawl Done")
f_posts.close()
session.close()

Crawl Start
	Reading page 169881 (0.00%)
	Reading page 169643 (0.05%)
	Reading page 169567 (0.10%)
	Reading page 169544 (0.15%)
	Reading page 169511 (0.20%)
	Reading page 169467 (0.25%)
	Reading page 169307 (0.30%)
	Reading page 169302 (0.35%)
	Reading page 169292 (0.40%)
	Reading page 169287 (0.45%)
	Reading page 169086 (0.50%)
	Reading page 169036 (0.55%)
	Reading page 169000 (0.60%)
	Reading page 168994 (0.65%)
	Reading page 168859 (0.70%)
	Reading page 168803 (0.75%)
	Reading page 168775 (0.80%)
	Reading page 168615 (0.85%)
	Reading page 168546 (0.90%)
	Reading page 168523 (0.95%)
	Reading page 168485 (1.00%)
	Reading page 168377 (1.05%)
	Reading page 168318 (1.10%)
	Reading page 168276 (1.15%)
	Reading page 168133 (1.20%)
	Reading page 168091 (1.25%)
	Reading page 168071 (1.30%)
	Reading page 168056 (1.35%)
	Reading page 167948 (1.40%)
	Reading page 167907 (1.45%)
	Reading page 167676 (1.50%)
	Reading page 167641 (1.55%)
	Reading page 167535 (1.60%)
	Reading page 167530 (1.65%)
	R

In [11]:
from google.colab import files
files.download("posts.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>